# Python introduction - session 4 
## Learning about the real power of python with modules
## Biopython, pandas, matplotlib and bedtools wrapped into the jupyter

If you get stuck in an endless loop hit the **"STOP" button (black square)** above or our good friend from bash, **ctrl+c**  
You know you are stuck in a loop if you see **In \[\*\]:** forever

### Installation reminder

If you want to install this whole tool stack on your own machine we recommend the following.

#### Windows

* Install the [Ubuntu subsystem](https://docs.microsoft.com/en-us/windows/wsl/install-win10)
* Install the Linux version of [Anaconda](https://www.anaconda.com/products/individual) into your subsystem.
* Setup [Bioconda](https://bioconda.github.io/) in your subsystem
* Install programs and modules like...

``conda install biopython``

#### Mac

* Install the Mac version of [Anaconda](https://www.anaconda.com/products/individual) on your command line/terminal.
* Setup [Bioconda](https://bioconda.github.io/) on your command line/terminal.
* Install programs and modules like...

``conda install biopython``

### Objectives

* Interogate larger dataset using biopython and pandas.
* Use pandas to combine larger datasets.
* Plot expression profiles across multiple condiditions in Arabidopsis.
* Extract promoter sequences of highly expressed genes.
* Analyze the composition of promoter sequences to identify motifs.

## Reminder: Python objects are like cats and dogs

In [ ]:
%matplotlib inline
from IPython.display import Image
import warnings
warnings.filterwarnings('ignore')

In [ ]:
Image(filename='./figures/dogs_as_objects.jpg')

### Intro to [Biopython](https://biopython.org/wiki/Documentation)

* We will download the complete [Arabidopsis](https://www.arabidopsis.org/index.jsp) coding sequences.
* We will read them all in at once.
* We will translate them all into protein sequences.
* We will calcualte their pi values.
* We will do some basic plotting.

In [ ]:
!gunzip datasets/TAIR10_chr_all.fa.gz

#### Let's download sequences from [here](https://www.arabidopsis.org/download/index-auto.jsp?dir=%2Fdownload_files%2FGenes%2FAraport11_genome_release%2FAraport11_blastsets)

In [ ]:
!wget https://www.arabidopsis.org/download_files/Genes/Araport11_genome_release/Araport11_blastsets/Araport11_genes.201606.cds.fasta.gz

In [ ]:
!gunzip Araport11_genes.201606.cds.fasta.gz

In [ ]:
!head Araport11_genes.201606.cds.fasta

## Let's reconstruct out lists from last time for all CDS in Arabidopsis

In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import IsoelectricPoint as IP
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
filename = 'Araport11_genes.201606.cds.fasta'

In [ ]:
## Read in all CDS as list (ordered!)
arabidopsis_cds = []
for seq in SeqIO.parse(filename, 'fasta'):
    arabidopsis_cds.append(seq)    

In [ ]:
###Let's make a cds length list and an ID list
cds_length = []
ID_list = []
for cds in arabidopsis_cds:
    cds_length.append(len(cds.seq))
    ID_list.append(cds.id)

In [ ]:
###Let's make a protein list and a protein length list
arabidopsis_proteins = []
protein_length = []
for cds in arabidopsis_cds:
    tmp_protein = cds.translate() #translate sequence
    tmp_protein.id = cds.id #add id to tmp_protein object
    tmp_protein.name = cds.name #add name to tmp_protein object 
    tmp_protein.description = cds.description #add description to tmp_protein object
    arabidopsis_proteins.append(tmp_protein) #store it
    protein_length.append(len(tmp_protein)) #store length

In [ ]:
protein_pi_values = []
for protein in arabidopsis_proteins:
    tmp_protein = IP.IsoelectricPoint(protein.seq) #generate a Isoelectric point object to be able to calcuate pi values
    tmp_protein_pi = tmp_protein.pi() #calculate pi value
    protein_pi_values.append(tmp_protein_pi)

## Exercise I

* Make a dictionary called ara_dict
* the keys 'CDS_ID', 'CDS_length', 'protein_length', 'protein_pi_value'
* the values are the corresponding lists we generated above

In [ ]:
ara_dict = {}
ara_dict['CDS_ID'] = ID_list
ara_dict['CDS_length'] = cds_length
ara_dict['protein_length'] = protein_length
ara_dict['protein_pi_value'] = protein_pi_values

### Intro to [pandas](https://pandas.pydata.org/)

pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language. 

* We will generate a dataframe (Table) from our own input data
* We will subset the dataframe to find the longest and shortes CDS in Arabidopsis
* We will caclulate the mean, median, and standard deviation for our dataset
* We will do some basic plotting.
* We will load some public expression datasets.
* We will interogate this dataset.
* We will pull out the promoters of the most highly expressed genes and look for a specific hormone response elements.

#### The two main objects in pandas 

* Series to store one dimensional data
* Dataframe to store two dimnesional data

See for a quick intro [here](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html)

In [ ]:
ara_df = pd.DataFrame.from_dict(ara_dict)

In [ ]:
ara_df

In [ ]:
## look at the top
ara_df.head()

In [ ]:
## look at the bottom
ara_df.tail()

In [ ]:
## Get the column index
ara_df.columns

In [ ]:
## Get row index
ara_df.index

In [ ]:
## Get specific columns
ara_df['CDS_ID']

In [ ]:
## Get a gene id column for later use
ara_df['Gene_ID'] = ara_df['CDS_ID'].apply(lambda x: x.split('.')[0])

In [ ]:
ara_df['Gene_ID']

In [ ]:
## Reset the index
ara_df.index = ara_df['Gene_ID']

In [ ]:
## Drop a column
ara_df.drop('Gene_ID', axis=1)

In [ ]:
## Drop a column inplace
ara_df.drop('Gene_ID', axis=1, inplace = True)

### A word about pandas axis

In [ ]:
Image(filename='./figures/axis_pandas.jpg')

### Getting the mean, max, min, and such from a dataset

In [ ]:
## maximum
ara_df['CDS_length'].max()

In [ ]:
## mean
ara_df['CDS_length'].mean()

### Exercise II

* Get the mean value of protein_length column
* Get the max value of the protein_length column
* Get the standard deviation of the protein lenght column
* Get the mean value of the protein_pi_value column

### Plotting is your friend and build into pandas

In [ ]:
##histogram plot of protein_pi_value
ara_df['protein_pi_value'].plot.hist(bins=500)

In [ ]:
##histogram plot with horizontal line of the 
ara_df['protein_pi_value'].plot.hist(bins=500)
plt.axvline(ara_df['protein_pi_value'].mean(), c='r')
plt.xlabel('PI value [pH]')

## Always look at your distributions genome data is never?! normally distributed

### Subsetting your data

In [ ]:
## CDS that are longer than the mean
ara_df['CDS_length'] > ara_df['CDS_length'].mean() 

In [ ]:
## Dataframe and Series have shapes
ara_df['CDS_length'].shape

In [ ]:
## Dataframe and Series have shapes
ara_df.shape

In [ ]:
(ara_df['CDS_length'] > ara_df['CDS_length'].mean()).shape

In [ ]:
## Because comparisons return the same shaped boolean series we can use them to subset dataframes
ara_df[ara_df['CDS_length'] > ara_df['CDS_length'].mean()]

In [ ]:
### We can also use boolean series to count
(ara_df['CDS_length'] > ara_df['CDS_length'].mean()).sum()

### Exercise III

* How many proteins are longer than 1000 aa.
* How many CDS are longer than the mean plus two standard deviations.
* What is the mean length of proteins longer than 2000 aa.

### Let's look at some public Arabidopsis gene expression data after Auxin (a hormone) treatment

The paper is about gene expression after two different hormone treatments in Arabidopsis. We will use this to learn more about pandas and to extract promoters in the next session.

[Spatiotemporal Brassinosteroid Signaling and Antagonism with Auxin Pattern Stem Cell Dynamics in Arabidopsis Roots](https://www.sciencedirect.com/science/article/pii/S0960982215002158#mmc2) 

The basic treatments we are interested in are in the Columbia (WT) background:

* Auxin alone.
* Brassinosteroid alone.
* Auxin and Brassinosteroid (but this wasn't really in the supplement).


In [ ]:
Image('./figures/Expression_analysis.png')

### This is how the data actually looks like

In [ ]:
Image('figures/Excel_screenshot.png')

### Let's read in this excel table into pandas

In [ ]:
?pd.read_excel

In [ ]:
## Excel file name
excel_fn = 'datasets/1-s2.0-S0960982215002158-mmc2.xlsx'

In [ ]:
### Read in excel file with specific spreadsheet and take care of excess row and index column
excel_df = pd.read_excel(excel_fn, sheet_name='S1', skiprows=1, index_col=0)

In [ ]:
###Let's look at it
excel_df.head()

### Let's look into indexing of rows and columns

In [ ]:
Image(filename='./figures/axis_pandas.jpg')

In [ ]:
## Numerical indexing (exclusive [,)])
excel_df.iloc[1,1]

In [ ]:
## Name indexing (inclusive [,])
excel_df.loc['AT3G54530', 'dwf4 +BL vs. dwf4']

In [ ]:
## Name indexing (inclusive [,])
excel_df.loc['AT3G54530':'AT5G20470', 'dwf4 +BL vs. dwf4']

In [ ]:
## Numerical indexing (exclusive [,)])
excel_df.iloc[1:2,1]

In [ ]:
### Let's subset the dataframe to only contain the columns
### 'Col+BL vs. Col', 'Col+IAA vs. Col', 'Computational_description'

excel_df = excel_df.loc[:, ['Col+BL vs. Col', 'Col+IAA vs. Col', 'Computational_description']]

In [ ]:
excel_df.head()

### Exercise V

* What is the differential expression after IAA treatment of gene 'AT3G46810'
* What is the maximal differential expression of BL treatment.
* What is the BL expression of the 1493 gene.

In [ ]:
###Double indexing "and"
excel_df[(excel_df['Col+BL vs. Col'] > 5) & (excel_df['Col+IAA vs. Col'] > 5)]

In [ ]:
###Double indexing "or"
excel_df[(excel_df['Col+BL vs. Col'] > 5) | (excel_df['Col+IAA vs. Col'] > 5)]

### Exercise VI

* How many genes are upregulated at both conditions.
* How many genes are upgregulated in one condition only.
* Do you think the overlap or exclusion is significant and what test could you use to test for this.


In [ ]:
###Sort values
excel_df.sort_values('Col+IAA vs. Col')

In [ ]:
###Sort values
excel_df.sort_values('Col+IAA vs. Col', ascending=False, inplace=True)

In [ ]:
###let's look at the top 100 differentially expressed (>5) expressed genes after IAA treatment
###that are also differentially expressed in BL conditions.
sub_excel_df = excel_df[(excel_df['Col+BL vs. Col'] > 5) | (excel_df['Col+IAA vs. Col'] > 5)].iloc[:100,:]

In [ ]:
sub_excel_df.head()

In [ ]:
###Sort values
sub_excel_df.plot.bar()

### Let's explore the data a bit more and ask the questions like

* What is the pi value distirubtion of BL upregulated genes.
* What is the length distribution of IAA upregulated genes.
* ....

Remember both the ara_df and excel_df have the gene names as index

In [ ]:
IAA_induced_index = excel_df[excel_df['Col+IAA vs. Col'] > 5].index

In [ ]:
ara_df.loc[ara_df.index.isin(IAA_induced_index),'protein_pi_value'].plot.kde(color='g', alpha=0.5)
ara_df['protein_pi_value'].plot.kde(color='b', alpha=0.5)

In [ ]:
ara_df.loc[ara_df.index.isin(IAA_induced_index),'protein_length'].plot.kde(color='g', alpha=0.5)
ara_df['protein_length'].plot.kde(color='b', alpha=0.5)

## Let's talk about genome annotations and coordinate files

### DNA sequences are 1 dimensional strings how to express what is encoded in a DNA sequence?

### [BED formats](https://m.ensembl.org/info/website/upload/bed.html)

* It's like a string that gets indexed in python starting with 0 and being exclusive **[,)**  
* Minimum we need is the sequence identifier, start, end -> **BED3**

```ctg123   1299    9000```

* Next up is we might want to encode the name, score, and strand -> **BED6**

```ctg123   1299    9000    mRNA    .    +```

### What about if we can to link different entries with each other e.g. genes -> mRNA -> UTRs/CDS -> exons?

### [GFF3 format](https://m.ensembl.org/info/website/upload/gff3.html)

* It's a coordinate system like BED but in a different order with more columns and 1 based being inclusive **[,]**

* SeqID, source, type, start, end, score, strand, phase, attributes

``` ##gff-version 3
ctg123 . mRNA            1300  9000  .  +  .  ID=mrna0001;Name=sonichedgehog
ctg123 . exon            1300  1500  .  +  .  ID=exon00001;Parent=mrna0001
ctg123 . exon            1050  1500  .  +  .  ID=exon00002;Parent=mrna0001
ctg123 . exon            3000  3902  .  +  .  ID=exon00003;Parent=mrna0001
ctg123 . exon            5000  5500  .  +  .  ID=exon00004;Parent=mrna0001
ctg123 . exon            7000  9000  .  +  .  ID=exon00005;Parent=mrna0001 ```

### Let's download the Arabidopsis gff3 file from [here](https://www.arabidopsis.org/download/index-auto.jsp?dir=%2Fdownload_files%2FGenes%2FTAIR10_genome_release%2FTAIR10_gff3)

The aim is to get all the rows that describe genes and that are part of our upregulated list.

In [ ]:
gff_header = ['SeqID', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']

In [ ]:
###read in csv
ara_gff3 = pd.read_csv

In [ ]:
###let's have a look


In [ ]:
###let's get the subset for 'AT1G01010' and 'type' being 'gene'
ara_gff3[(ara_gff3) & (ara_gff3)]

In [ ]:
### Let's get the index 


### Remaining Lesson

* get all the Auxin (IAA) upregulated genes above 5 fold up.
* get the gene entries out of the gff file for the upregulated genes and the none-regualted genes
* get the promoters for both sets in bedformat and in fasta format
* look for the previously characterized elments **TGTCCCAT** or **GGTCCCAT** which have been shown previously to be involved in auxin (IAA) responsiveness

In [ ]:
###Let's get the 'Col+IAA vs. Col' upregulated genes from the excel_df
IAA_genes = excel_df[]

In [ ]:
###Let's safe them sorted
IAA_genes = IAA_genes.sort_values()

In [ ]:
###Let's try to collect all the indexes (the)
IAA_gene_index = []
IAA_genes_annotated = []
###Let's make the dateset smaller
ara_gene_gff3 = ara_gff3[]
##Let's reset the index
ara_gene_gff3.


In [ ]:
###Let's loop over the dataset and get all the indexes for genes we want
for gene in IAA_genes:
    tmp_index = 
    #if the length of an index is 0 there was no element found fitting the bill
    if :
        print('Gene not found in annotation file:', gene)
    
    elif :
        
    else:
        print('Multiple annotations found for the same gene:', gene)
        

In [ ]:
###Let's use he indexes to get the gene gff3 of all the genes of interest
ara_gene_gff3.loc[##]

In [ ]:
###Let's save it out with to_csv
ara_gene_gff3.loc[##]

### exercise I

* Find out how many upregulated IAA genes there are.
* Find out how many upregulated IAA genes where in the annotation.



### exercise II

* safe out the gff3 file in bed six format
    * Generate the correct start side for bed format being 0 based.
    * Generate a new gene name column
    * Fill in the names for the IAA genes that were annotated IAA_genes_annotated
    * Change the orientation of the columns to 'SeqID', 'Bed_start', 'end', 'name', 'score', 'strand'


In [ ]:
ara_gene_gff3['Bed_start'] = ### - 1
ara_gene_gff3[###] = 'NaN'
ara_gene_gff3.loc[IAA_gene_index, 'name'] = ######

In [ ]:
###save out in the right order for our select genes only
ara_gene_gff3.loc[#####].to_csv('datasets/IAA_up.gene.bed6', ####)

In [ ]:
###Let's safe out all the gene beds as well.
ara_gene_gff3.loc[#####].to_csv('datasets/All.gene.bed6', ####)

## A crash course introduction to [Bedtools](https://bedtools.readthedocs.io/en/latest/index.html)

Collectively, the bedtools utilities are a swiss-army knife of tools for a wide-range of genomics analysis tasks. The most widely-used tools enable genome arithmetic: that is, set theory on the genome.

* We want to get all genes that are not upregulated.

-> Upregulated genes **(IAA_up.gene.bed6)** and not upregulated genes **(noIAA_up.gene.bed6)**

## Inverse a bed file with bedtools [subtract](https://bedtools.readthedocs.io/en/latest/content/tools/subtract.html?highlight=subtract)

In [ ]:
Image('figures/Substract.png')

This is a quick way to inverse a gene bed file and we want to work on both.


In [ ]:
!bedtools subtract ### > datasets/noIAA_up.gene.bed6

In [ ]:
!bedtools subtract

### Now we have two gene beds we want to work with 

**(IAA_up.gene.bed6)** and not upregulated genes **(noIAA_up.gene.bed6)**

## Get the promoters with bedtools [flank](https://bedtools.readthedocs.io/en/latest/content/tools/flank.html?highlight=flank)

In [ ]:
Image('figures/Flank.png')

In [ ]:
!bedtools flank ##### > datasets/IAA_up.promoter.bed6 

In [ ]:
!bedtools flank

### exercise III
Generate the promoters for the noIAA_up.gene.bed6 file

In [ ]:
!bedtools flank ##### > datasets/noIAA_up.promoter.bed6 

## Get the promoters with bedtools [getfasta](https://bedtools.readthedocs.io/en/latest/content/tools/getfasta.html?highlight=getfasta)

In [ ]:
Image('figures/Getfasta.png')

In [ ]:
!bedtools getfasta #### >  datasets/IAA_up.promoter.fa

In [ ]:
!bedtools getfasta

### exercise IV
Get the fasta sequence for the noIAA_up.promotor.bed6 file

In [ ]:
!bedtools getfasta #### >  datasets/noIAA_up.promoter.fa

### Back to biopython to count the Auxin/IAA features in promoters

In [ ]:
IAA_pro_motif_counter = 0
IAA_pro_counter = 0
for #######:
    IAA_pro_counter += 1
    if #######:
        IAA_pro_motif_counter += 1

In [ ]:
print('There are', IAA_pro_motif_counter, 'out of', IAA_pro_counter, \
      'upregulated IAA gene promoters which contain one of the two IAA sequences.')
print('This is', IAA_pro_motif_counter/IAA_pro_counter*100, '%.')

### exercise V
Count the promoter sequences of genes not upregualted by IAA and how many contain the motifs

In [ ]:
noIAA_pro_motif_counter = 0
noIAA_pro_counter = 0
for pro in #####:
    noIAA_pro_counter += 1
    if ####:
        noIAA_pro_motif_counter += 1

In [ ]:
print('There are', noIAA_pro_motif_counter, 'out of', noIAA_pro_counter, \
      'not upregulated IAA gene promoters which contain one of the two IAA sequences.')
print('This is', noIAA_pro_motif_counter/noIAA_pro_counter*100, '%.')

### What is your conclusion?